In [1]:
import pandas as pd
from datetime import datetime

In [8]:
df = pd.read_csv("../data/SG-ES.csv", encoding="utf-8", sep=";")
df[:5]

,sintomas,profissionalSaude,racaCor,outrosSintomas,outrasCondicoes,profissionalSeguranca,cbo,condicoes,sexo,estado,...,codigoResultadoTeste3,codigoEstadoTeste4,codigoTipoTeste4,codigoFabricanteTeste4,codigoResultadoTeste4,dataColetaTeste1,dataColetaTeste2,dataColetaTeste3,dataColetaTeste4,idade
0,Assintomático,Não,Ignorado,NaN,NaN,Não,NaN,NaN,Feminino,São Paulo,...,NaN,NaN,NaN,NaN,NaN,2022-02-16,NaN,NaN,NaN,38.0
1,Assintomático,Não,Parda,NaN,NaN,Não,NaN,NaN,Masculino,Pernambuco,...,NaN,NaN,NaN,NaN,NaN,2022-06-25,NaN,NaN,NaN,47.0
2,"Coriza, Tosse, Dor de Cabeça",Não,Ignorado,NaN,NaN,Não,NaN,NaN,Feminino,São Paulo,...,NaN,NaN,NaN,NaN,NaN,2022-06-21,NaN,NaN,NaN,NaN
3,Assintomático,Não,Ignorado,NaN,NaN,Não,NaN,NaN,Feminino,São Paulo,...,NaN,NaN,NaN,NaN,NaN,2022-07-30,NaN,NaN,NaN,NaN
4,Assintomático,Não,Parda,NaN,NaN,Não,NaN,NaN,Feminino,Pernambuco,...,NaN,NaN,NaN,NaN,NaN,2022-06-28,NaN,NaN,NaN,NaN


In [ ]:

date_range = pd.date_range(
    # start=df['dataInicioSintomas'].min(), Notificação ou início dos sintomas?
    start=df['dataNotificação'].min(),
    end=df['dataEncerramento'].max(),
    freq='D'
)

sir = pd.DataFrame(index=date_range, columns=['S', 'I', 'R'])

In [ ]:

# Discarding non recovery/death outcomes
df = df[df["Evolucao"].isin(["Cura", "Óbito pelo COVID-19"])]

# Translating column names
df["NotificationDate"] = df["dataNotificação"]
df["RecoveryDate"] = df["DataObito"]
df["RecoveryDate"] = df.apply(lambda row: row["DataEncerramento"] if row["Evolucao"] == "Cura" else None, axis=1)

# Converting to datetime
df["NotificationDate"] = pd.to_datetime(df["NotificationDate"], format='%Y-%m-%d')
df["DeathDate"] = pd.to_datetime(df["DeathDate"], format='%Y-%m-%d')
df["RecoveryDate"] = pd.to_datetime(df["RecoveryDate"], format='%Y-%m-%d')

# Discarding inconsistent entries
df = df[df["RecoveryDate"] >= df["NotificationDate"]]


In [ ]:

def filter_csv_by_time_period(input_file, output_file, time_column, start_time, end_time, chunksize=100000):
    """
    Filters a massive CSV file for rows within a specified time period.
    
    Parameters:
        input_file (str): Path to the input CSV file
        output_file (str): Path to save the filtered CSV
        time_column (str): Name of the column containing timestamps
        start_time (str): Start time in format matching your CSV (e.g., '2023-01-01')
        end_time (str): End time in format matching your CSV (e.g., '2023-01-31')
        chunksize (int): Number of rows to process at a time (adjust based on memory)
    """
    
    # Convert string times to datetime objects for comparison
    start_dt = pd.to_datetime(start_time)
    end_dt = pd.to_datetime(end_time)
    
    # Initialize a list to hold filtered chunks
    filtered_chunks = []
    
    # Read the CSV in chunks
    for chunk in pd.read_csv(input_file, parse_dates=[time_column], chunksize=chunksize, encoding="latin-1", sep=";"):
        # Filter the current chunk
        mask = (chunk[time_column] >= start_dt) & (chunk[time_column] <= end_dt)
        filtered_chunk = chunk.loc[mask]
        
        # Append to our list of filtered chunks
        if not filtered_chunk.empty:
            filtered_chunks.append(filtered_chunk)
    
    # Combine all filtered chunks and save to CSV
    if filtered_chunks:
        pd.concat(filtered_chunks).to_csv(output_file, index=False)
        print(f"Filtered data saved to {output_file}")
    else:
        print("No data found within the specified time period.")

# Example usage
if __name__ == "__main__":
    input_csv = "MICRODADOS.csv"          # Replace with your input file
    output_csv = "sir-es.csv"      # Replace with your output file
    time_col = "DataNotificacao"                # Replace with your time column name
    start = "2020-01-01 00:00:00"        # Replace with your start time
    end = "2021-01-31 23:59:59"          # Replace with your end time
    
    filter_csv_by_time_period(input_csv, output_csv, time_col, start, end)

/tmp/ipykernel_360155/227377149.py:22: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(input_file, parse_dates=[time_column], chunksize=chunksize, encoding="latin-1", sep=";"):
/tmp/ipykernel_360155/227377149.py:22: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(input_file, parse_dates=[time_column], chunksize=chunksize, encoding="latin-1", sep=";"):
/tmp/ipykernel_360155/227377149.py:22: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(input_file, parse_dates=[time_column], chunksize=chunksize, encoding="latin-1", sep=";"):
/tmp/ipykernel_360155/227377149.py:22: DtypeWarning: Columns (5,6,8) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(input_file, parse_dates=[time_column], chunksize=chunksize

Filtered data saved to covid-es.csv
